# Desafio Titanic Kaggle

### Anderson Rocha
#### http://scikit-learn.org/
#### https://www.kaggle.com/c/titanic/data

### Importanto dados

In [49]:
import numpy as np
import pandas as pd
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
import matplotlib.pyplot as plt

from __future__ import print_function

%matplotlib inline

attributes = ['PassengerId', 'Survived', 'Pclass', 'Name',
              'Sex', 'Age', 'SibSp', 'Parch', 'Ticket',
              'Fare', 'Cabin', 'Embarked']

features = [a for a in attributes if a != 'Survived']


# Treino
train = pd.read_csv('train.csv', header=0, names=attributes, delimiter=',')

# Test
test = pd.read_csv('test.csv', header=0, names=features, delimiter=',')

### Analisando dados

In [61]:
print(train.head())
print("----------------------------------------------------------------------------")
print(train.describe())
print("-----------------------------Verificando dados nulos------------------------")

print(train.groupby('Survived')['PassengerId'].nunique())

print("----------------------------------------------------------------------------")

for attr in attributes:
    y = train[train[attr].isnull()]
    nulos = len(y)
    if nulos > 0:
        print(f"O atributo {attr} tem {nulos} nulos")
    

   PassengerId  Survived  Pclass  \
0            1         0       3   
1            2         1       1   
2            3         1       3   
3            4         1       1   
4            5         0       3   

                                                Name     Sex   Age  SibSp  \
0                            Braund, Mr. Owen Harris    male  22.0      1   
1  Cumings, Mrs. John Bradley (Florence Briggs Th...  female  38.0      1   
2                             Heikkinen, Miss. Laina  female  26.0      0   
3       Futrelle, Mrs. Jacques Heath (Lily May Peel)  female  35.0      1   
4                           Allen, Mr. William Henry    male  35.0      0   

   Parch            Ticket     Fare Cabin Embarked  
0      0         A/5 21171   7.2500   NaN        S  
1      0          PC 17599  71.2833   C85        C  
2      0  STON/O2. 3101282   7.9250   NaN        S  
3      0            113803  53.1000  C123        S  
4      0            373450   8.0500   NaN        S  
--

* Podemos ver que temos um maior número de mortes, (classe 0 mortos, 1 vivos). De **891** registros, temos **549** mortos. Isso pode ser um problema para o nosso algoritmo, pois a classe majoritária pode ser favorecida no treino.

* Temos mais um problema que é a cabine em que os passageiros estavam. Temos 687 registros com valores nulos **(687)**.
* Outro problema que temos é que temos muitos registro com o atributo idade nulo **(177)**.
* Além desses problemas temos **2** registros de embarcado com null, mais não é um número preocupante neste momento.

### Tratando dados de treino

Vamos descobrir qual a melhor maneira de inferir as idades nulas. Podemos colocar a média para todos os registros nulos ou podemos inferir através de outros atributos.

Observando os atributos, vemos que o nome das pessoas sempre tem as iniciais (Miss, Mr, etc), então podemos agrupar as médias por essas iniciais.

In [90]:
names_and_ages = train[["Name", "Age"]]
list_tuples_with_initials_and_names = []
for i in range(1, len(names_and_ages["Name"])):
    name = names_and_ages["Name"][i]
    age = names_and_ages["Age"][i]
    splitted_name = name.split(",")
    splitted_name_inicial = splitted_name[1].split(".")
    list_tuples_with_initials_and_names.append((splitted_name_inicial[0], age))
    
df_initials_and_names = pd.DataFrame(list_tuples_with_initials_and_names, columns=["Initial", "Age"])

print(df_initials_and_names.groupby("Initial").agg("mean"))

                     Age
Initial                 
 Capt          70.000000
 Col           58.000000
 Don           40.000000
 Dr            42.000000
 Jonkheer      38.000000
 Lady          48.000000
 Major         48.500000
 Master         4.574167
 Miss          21.773973
 Mlle          24.000000
 Mme           24.000000
 Mr            32.394207
 Mrs           35.898148
 Ms            28.000000
 Rev           43.166667
 Sir           49.000000
 the Countess  33.000000


### Selecionando os melhores atributos

### Oversampling - adicionando dados fake a classe minoritária

### Validação cruzada de algoritmos e hiper parâmetros